In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
train = train.drop(columns=['PassengerId', 'Name'])
y_train = train.pop('Survived')
test_id = test.pop('PassengerId')
test = test.drop(columns='Name')

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Ticket    891 non-null    object 
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 62.8+ KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Ticket    418 non-null    object 
 6   Fare      417 non-null    float64
 7   Cabin     91 non-null     object 
 8   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 29.5+ KB


In [9]:
train['Age'] = train['Age'].fillna(train['Age'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())

In [10]:
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [11]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [13]:
train_dum = pd.get_dummies(train)

In [14]:
x_tr, x_val, y_tr, y_val = train_test_split(train_dum, y_train, test_size=0.2, stratify=y_train, random_state=0)

In [83]:
model = RandomForestClassifier(random_state=0, max_depth=25)

In [90]:
train_dum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male',
       'Ticket_110152', 'Ticket_110413', 'Ticket_110465',
       ...
       'Cabin_F G73', 'Cabin_F2', 'Cabin_F33', 'Cabin_F38', 'Cabin_F4',
       'Cabin_G6', 'Cabin_T', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object', length=838)

In [84]:
model.fit(x_tr, y_tr)

RandomForestClassifier(max_depth=25, random_state=0)

In [85]:
pred = model.predict_proba(x_val)[:, 1]
roc_auc_score(y_val, pred)

0.8482213438735178

In [ ]:
# 0.8316205533596839
# 0.8391963109354414

In [18]:
numeric_cols = train.select_dtypes(exclude='O').columns.to_list()

In [19]:
numeric_cols

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [20]:
scaler = RobustScaler()

In [21]:
for col in numeric_cols:
    train[col] = scaler.fit_transform(train[[col]])
    test[col] = scaler.transform(test[[col]])

In [22]:
train1 = train.drop(columns='Cabin')
train2 = train.copy()

In [24]:
train1 = pd.get_dummies(train1)
train2 = pd.get_dummies(train2)

In [29]:
x_tr1, x_val1, y_tr1, y_val1 = train_test_split(train1, y_train, test_size=0.2, stratify=y_train, random_state=0)
x_tr2, x_val2, y_tr2, y_val2 = train_test_split(train2, y_train, test_size=0.2, stratify=y_train, random_state=0)

In [86]:
model1 = RandomForestClassifier(random_state=0, max_depth=25)
model2 = RandomForestClassifier(random_state=0, max_depth=25)

In [87]:
model1.fit(x_tr1, y_tr1)
model2.fit(x_tr2, y_tr2)

RandomForestClassifier(max_depth=25, random_state=0)

In [88]:
pred1 = model1.predict_proba(x_val1)[:, 1]
pred2 = model2.predict_proba(x_val2)[:, 1]

In [89]:
roc_auc_score(y_val1, pred1), roc_auc_score(y_val2, pred2)

(0.845125164690382, 0.847957839262187)